In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import pandas as pd
import json
from pathlib import Path
import deepdish as dd

from matplotlib import pyplot as plt
import seaborn as sns

sns.set(style="ticks", palette="deep")

In [ ]:
def load_from_path(path):
    """Utility function to load Stytra metadata and log for an experiment.
    """
    
    # Open json:
    with open(next(path.glob("*.json*")), "r") as f:
        metadata_dict = json.load(f)
        
    # Open log:
    beh_df = dd.io.load(next(fish_path.glob("*behavior*")))["data"]
    beh_df = beh_df.set_index(["t"])  # set time as index
    
    return metadata_dict, beh_df

## Calculate image scale

Compute pixel size given size of the arena in pixels and in mms:

In [ ]:
# Numbers taken from image size and physical setup size:
arena_size_mm = 35
arena_size_pixels = 770

mm_pixel = arena_size_mm / arena_size_pixels

## Find fish folders:

In [ ]:
folder_name = r"...\free_swimming_protocol" # Folder with Stytra fish sub-directories

# Find list with all fish directories:
master_path = Path(folder_name)
fish_paths = list(master_path.glob("*f[0-9]"))

print("Folders: \n" + "\n".join([str(f)for f in fish_paths]))

## Make figure for example fish

Create and save trajectory plot for fish i from the list above:

In [ ]:
i = 0 # change number here to select another fish

fish_path = fish_paths[i]  
metadata, beh_df = load_from_path(fish_path)
print("loading fish {}...".format(fish_path.name))

In [ ]:
f, ax = plt.subplots(figsize=(5, 5))

bar_length_mm = 10

ax.plot(beh_df["f0_x"] * mm_pixel, beh_df["f0_y"] * mm_pixel)
ax.axis("equal")
ax.set_yticks([])
ax.set_xticks([0, bar_length_mm])
ax.set_xticklabels([])
ax.set_xticks([bar_length_mm / 2], minor=True)
ax.set_xticklabels(["{} mm".format(bar_length_mm)], minor=True)
ax.tick_params(length=0, which="both")
ax.set_title(fish_path.name)

sns.despine(left=True, trim=True)
f.savefig(str(fish_path / "trajectory_plot.pdf"), format="pdf")


# Extract distance travelled from all fish in folder

In [ ]:
smooth_win_s = 0.030  # 30 ms window used to smooth data before calculating speed

distances_travelled = []

for fish_path in master_path.glob("*f[0-9]"):
    print("Processing {}...".format(fish_path.name))
    metadata, beh_df = load_from_path(fish_path)
    dt = np.median(np.diff(beh_df.index))  # find timepoint duration in sec

    # Smooth the velocities:
    smooth_df = beh_df.rolling(int(smooth_win_s / dt), center=True).mean()

    # Calculate speed:
    speed = np.sqrt(smooth_df["f0_vx"] ** 2 + smooth_df["f0_vy"] **2)

    distances_travelled.append(np.sum(speed) * mm_pixel)

# Create and save DataFrame:
pd.DataFrame(np.array(distances_travelled), columns=["distance (mm)"],
             index=[f.name for f in fish_paths]).to_excel(master_path / "all_distances.xlsx")